# Import Packages

In [37]:
import nltk
from nltk.corpus import wordnet
# nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
import numpy as np
import pandas as pd
import random
import re 
import string
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
from spacy.lang.en import English

from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD 
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.model_selection import GridSearchCV, train_test_split

# Plotting tools
import pyLDAvis
import pyLDAvis.sklearn
import seaborn
import matplotlib.pyplot as plt
%matplotlib inline

# Load Data

In [11]:
talk_df = pd.read_csv('has_transcript_clean.csv', index_col = 0)

In [12]:
talk_df.head()

,date,speaker,title,url,length,summ,tags,views,transcript,date_recorded,upload_date,occupation,bio,comments,duration
0,Jan 2020,Ipsita Dasgupta,"To challenge the status quo, find a ""co-conspi...",/talks/ipsita_dasgupta_to_challenge_the_status...,11:03,"In a complex and changing world, how can we ma...","['innovation', 'collaboration', 'society', 'so...",599446.0,So I've been thinking about how to explain thi...,2019-09-24,2020-01-02,"Business executive, ""co-conspirator""",Ipsita Dasgupta drives the consumption of ente...,7.0,663
1,Jan 2020,Rod Phillips,A brief history of alcohol,/talks/rod_phillips_a_brief_history_of_alcohol,4:56,Nobody knows exactly when humans began to crea...,"['TED-Ed', 'education', 'animation', 'history'...",501290.0,This chimpanzee stumbles across a windfall of ...,2020-01-02,2020-01-02,NaN,NaN,NaN,296
2,Jan 2020,Pat Mitchell,Dangerous times call for dangerous women,/talks/pat_mitchell_dangerous_times_call_for_d...,17:14,Pat Mitchell has nothing left to prove and muc...,"['women', 'women in business', 'community', 'a...",461705.0,"Recently, I've been declaring to anyone who wo...",2019-12-04,2020-01-02,Dangerous woman,Pat Mitchell is a lifelong advocate for women ...,14.0,1034
3,Dec 2019,Cara E. Yar Khan,The beautiful balance between courage and fear,/talks/cara_e_yar_khan_the_beautiful_balance_b...,9:55,After being diagnosed with a rare genetic cond...,"['fear', 'personal growth', 'health', 'life', ...",880662.0,"When we're young, we're innocently brave, and ...",2019-12-04,2019-12-23,Human rights and disability activist,Cara E. Yar Khan is an international human rig...,17.0,595
4,Dec 2019,Valorie Kondos Field,Why winning doesn't always equal success,/talks/valorie_kondos_field_why_winning_doesn_...,15:49,Valorie Kondos Field knows a lot about winning...,"['success', 'sports', 'leadership', 'empathy',...",956498.0,"OK, I have a question for all of us. You ready...",2019-12-04,2019-12-20,Gymnastics coach,Valorie Kondos Field is the retired head coach...,18.0,949


# Create Tokenizer

In [13]:
# Create our list of punctuation marks
punctuations = string.punctuation

# Create our list of stopwords
nlp = spacy.load('en_core_web_sm')
stop_words = spacy.lang.en.stop_words.STOP_WORDS

def spacy_tokenizer(text):
    
    # Remove audience reactions -- need to revisit (remove )
    no_audience_reacts = text.replace('(Applause)', '').replace('(Laughter)', '').replace('(applause)', '').replace('(laughter)', '')

    # Remove quotation marks
    no_quotes = no_audience_reacts.replace('\"', ' ').replace('”', ' ').replace('’', '')
    
    # Address hyphenation issue -- need to revisit
    no_ism = no_quotes.replace('-ism', 'ism')
    no_dash = no_ism.replace('–', ' ').replace('-', '').replace('—', ' ')
    
    # Remove parentheses
    no_parentheses = no_dash.replace('(', ' ').replace(')', ' ')
    
    # Remove .., ..., ....
    no_ellipses = no_parentheses.replace('....', '').replace('...', '').replace('..','').replace('…', '')
    
    # Remove music notes
    no_notes = no_ellipses.replace('♪', '').replace('♫', '')
    
    # HANDLE NUMBERS!
    
    # If ' ' then any alphanumeric then punctuation then any capitalized word then ' ' (correct for missing space after end of sentence)
    
    # Replace all whitespace with one space
    cleantext = ' '.join(no_notes.split())
    cleantext = cleantext.strip()
    
    # Creating our token object, which is used to create documents with linguistic annotations.
    # we disabled the parser and ner parts of the pipeline in order to speed up parsing
    mytokens = nlp(cleantext.lower(), disable=['parser', 'ner'])

    # Removing words except for nouns, verbs, adjectives, and adverbs
    mytokens = [word for word in mytokens if (word.pos_ == 'NOUN') or (word.pos_ == 'VERB') or (word.pos_ == 'ADJ') or (word.pos_ == 'ADV')]
    
    # Lemmatizing each token and converting each token into lowercase
    mytokens = [word.lemma_.strip() for word in mytokens]

    # Removing stop words
    mytokens = [word for word in mytokens if word not in stop_words and word not in punctuations]

    # return preprocessed list of tokens
    return mytokens

In [36]:
lemmatizer = WordNetLemmatizer()

# Test Tokenizer on Sentence-level and Speech-level

In [15]:
sentence_1 = "So we knew the numbers were deplorable, and it was manifesting itself in our own organization.So we decided — actually, I\'ll just give you a quick example of a young engineer, a 25-year-old woman, who told us a great story about her daily life, to just exemplify it for us.People, people, place, person, personally."

In [8]:
lemmatizer.lemmatize(sentence_1)

"So we knew the numbers were deplorable, and it was manifesting itself in our own organization.So we decided — actually, I'll just give you a quick example of a young engineer, a 25-year-old woman, who told us a great story about her daily life, to just exemplify it for us.People, people, place, person, personally."

In [306]:
# remove html tags from all of the text before processing
cleanr = re.compile('[.!?,][A-Z][A-Za-z]+ ')
cleantext = re.search(cleanr, sentence_1)

In [307]:
cleantext.group(0)

'.So '

In [16]:
transcript_1 = talk_df.transcript[0]

In [18]:
tokenized = spacy_tokenizer(sentence_1)

In [21]:
tokenized

['know',
 'number',
 'deplorable',
 'manifest',
 'organization.so',
 'decide',
 'actually',
 'quick',
 'example',
 'young',
 'engineer',
 '25yearold',
 'woman',
 'tell',
 'great',
 'story',
 'daily',
 'life',
 'exemplify',
 'people',
 'place',
 'person',
 'personally']

In [310]:
cv_test = CountVectorizer(tokenizer=spacy_tokenizer, min_df=0, stop_words='english')

In [311]:
sentence_cv = cv_test.fit_transform([sentence_1, transcript_1])

In [312]:
cv_test.get_feature_names()

['20',
 '25yearold',
 '40',
 'able',
 'absorb',
 'accent',
 'accountant',
 'achieve',
 'activity',
 'actually',
 'age',
 'allow',
 'american',
 'analysis',
 'annual',
 'apprehensive',
 'aside',
 'ask',
 'baby',
 'bank',
 'believe',
 'bend',
 'big',
 'bit',
 'boardroom',
 'bomb',
 'brace',
 'break',
 'brilliant',
 'bring',
 'building',
 'bunch',
 'bus',
 'business',
 'buy',
 'buyin',
 'calcutta',
 'care',
 'career',
 'ceo',
 'challenge',
 'champion',
 'change',
 'child',
 'chore',
 'class',
 'close',
 'coconspirator',
 'coconspire',
 'colleague',
 'come',
 'communicate',
 'commute',
 'company',
 'completely',
 'complex',
 'concept',
 'conspire',
 'context',
 'conventional',
 'conversation',
 'corporate',
 'couple',
 'courage',
 'create',
 'crystallize',
 'cup',
 'cure',
 'customer',
 'cute',
 'd',
 'daily',
 'daughtersinlaw',
 'day',
 'decade',
 'decide',
 'deplorable',
 'desert',
 'different',
 'disadvantage',
 'disease',
 'distinct',
 'dry',
 'dynamic',
 'early',
 'earn',
 'easy',
 'e

# Create Vectorizers
### Considerations
* Which vectorizer?
    * CountVectorizer
    * TfidfVectorizer
* Which topic modeling technique?
    * LSA (latent semantic analysis)
    * LDA (latent dirichlet allocation
* How many times can a token appear in the corpus?
    * Start: max_df = 0.9
    * Used:
        * 0.5
* How many tokens to include?
    * Start: all
    * Used:
        * 1000
        * 10000 --> best so far, but way too many as a result (with nouns, verbs, adj, adv)
* How many topics to use?
    * Used:
        * 25
        * 50
* How to evaluate topics


* Issue with lemmatization
* Issue with hyphens
* Consider numbers --> only if n-grams
* Issue with periods
* What is the most common word? for longer speeches?
* Default arguments

In [345]:
cv = CountVectorizer(tokenizer=spacy_tokenizer, max_df=0.50, min_df=3, stop_words='english')
tfidf = TfidfVectorizer(tokenizer = spacy_tokenizer, max_features=10000, min_df=3, max_df=0.50)

# Subset for data with views

In [11]:
talk_views = talk_df[talk_df.views.notnull() & talk_df.views > 0]

In [13]:
talk_views.shape

(3599, 15)

In [14]:
talk_views.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3599 entries, 0 to 3648
Data columns (total 15 columns):
date             3599 non-null object
speaker          3599 non-null object
title            3599 non-null object
url              3599 non-null object
length           3599 non-null object
summ             3599 non-null object
tags             3599 non-null object
views            3599 non-null float64
transcript       3599 non-null object
date_recorded    3510 non-null object
upload_date      3510 non-null object
occupation       3181 non-null object
bio              3193 non-null object
comments         2982 non-null float64
duration         3599 non-null int64
dtypes: float64(2), int64(1), object(12)
memory usage: 449.9+ KB


# Train, Test, Split

In [16]:
X = talk_views.transcript
y = talk_views.views

In [17]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42)

# Preprocess Transcripts

### CountVectorizer

In [99]:
# fit and transform training data
dtm_train_cv = cv.fit_transform(X_train)
dtm_test_cv = cv.transform(X_test)

In [100]:
len(cv.get_feature_names())

18530

In [101]:
dtm_train_cv_df = pd.DataFrame(dtm_train_cv.toarray())
dtm_train_cv_df.columns = cv.get_feature_names()

In [181]:
cv.get_feature_names()

NotFittedError: Vocabulary not fitted or provided

In [186]:
dtm_train_cv_df.columns[8000:8100]

Index(['ideally', 'ideas.and', 'ideas.now', 'ideas.so', 'ideation',
       'identical', 'identically', 'identifiable', 'identification',
       'identifier', 'identify', 'identity', 'identity.now', 'ideo',
       'ideological', 'ideologically', 'ideologue', 'ideology',
       'idiosyncratic', 'idiot', 'idiotic', 'idle', 'idleness', 'idly', 'idol',
       'idyllic', 'ied', 'ifs', 'igloo', 'ignite', 'ignition', 'ignorance',
       'ignorant', 'ignore', 'iguana', 'ikea', 'iliad', 'ill', 'illegal',
       'illegally', 'illegitimate', 'illicit', 'illiteracy', 'illiterate',
       'illness', 'illogical', 'illuminate', 'illuminating', 'illumination',
       'illusion', 'illusionist', 'illusory', 'illustrate', 'illustration',
       'illustrator', 'illustrious', 'image', 'image.so', 'imagery',
       'imaginable', 'imaginary', 'imagination', 'imagination.so',
       'imagination.thank', 'imaginative', 'imaginatively', 'imagine',
       'imagine.thank', 'imagine.this', 'imaging', 'imagining', '

## TfidfVectorizer

In [346]:
# fit and transform training data
dtm_train_tfidf = tfidf.fit_transform(X_train)
dtm_test_tfidf = tfidf.transform(X_test)

In [347]:
len(tfidf.get_feature_names())

10000

In [348]:
dtm_train_tfidf_df = pd.DataFrame(dtm_train_tfidf.toarray())
dtm_train_tfidf_df.columns = tfidf.get_feature_names()

In [349]:
dtm_train_tfidf_df.columns

Index(['10th', '11th', '11yearold', '15th', '15yearold', '1600', '1700',
       '17th', '1800', '1930',
       ...
       'zillion', 'zinc', 'zip', 'zipcar', 'zipper', 'zombie', 'zone',
       'zoning', 'zoo', 'zoom'],
      dtype='object', length=10000)

# Fit TruncatedSVD (LSA)

## CountVectorizer

In [109]:
lsa_cv = TruncatedSVD(n_components = 25, algorithm = 'randomized', n_iter = 3, random_state = 42)

In [110]:
lsa_cv.fit(dtm_train_cv)

TruncatedSVD(algorithm='randomized', n_components=25, n_iter=3, random_state=42,
             tol=0.0)

In [111]:
for index,topic in enumerate(lsa_cv.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([cv.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['kid', 'important', 'love', 'understand', 'man', 'build', 'learn', 'percent', 'brain', 'country', 'child', 'question', 'problem', 'story', 'woman']


THE TOP 15 WORDS FOR TOPIC #1
['friend', 'boy', 'kid', 'community', 'mother', 'young', 'love', 'family', 'school', 'country', 'girl', 'child', 'story', 'man', 'woman']


THE TOP 15 WORDS FOR TOPIC #2
['experience', 'sound', 'cancer', 'memory', 'girl', 'study', 'music', 'sleep', 'love', 'neuron', 'body', 'man', 'cell', 'woman', 'brain']


THE TOP 15 WORDS FOR TOPIC #3
['problem', 'government', 'dollar', 'money', 'drug', 'care', 'cancer', 'patient', 'disease', 'child', 'cell', 'health', 'percent', 'country', 'brain']


THE TOP 15 WORDS FOR TOPIC #4
['sound', 'parent', 'teach', 'love', 'education', 'brain', 'student', 'teacher', 'game', 'learn', 'music', 'play', 'school', 'child', 'kid']


THE TOP 15 WORDS FOR TOPIC #5
['design', 'learn', 'community', 'grow', 'family', 'cancer', 'building', 'city', 'food', 'lig

In [112]:
# Create Document - Topic Matrix
lsa_output = lsa_cv.transform(dtm_train_cv)

# column names
topicnames = ["Topic" + str(i) for i in range(lsa_cv.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(X_train))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lsa_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [113]:
unique, counts = np.unique(dominant_topic, return_counts=True)

In [114]:
print(np.asarray((unique, counts)).T)

[[   0 2535]
 [   1   65]
 [   2   72]
 [   3    2]
 [   4   12]
 [   5    3]
 [   6   44]
 [   7    9]
 [   8   18]
 [   9   71]
 [  10    8]
 [  11    1]
 [  12    1]
 [  13    3]
 [  14    2]
 [  15    5]
 [  16    1]
 [  17    4]
 [  18    1]
 [  20    4]
 [  21    3]
 [  22    5]
 [  23    6]
 [  24    4]]


## TfidfVectorizer

In [350]:
lsa_tfidf = TruncatedSVD(n_components = 25, algorithm = 'randomized', n_iter = 3, random_state = 42)

In [351]:
lsa_tfidf.fit(dtm_train_tfidf)

TruncatedSVD(algorithm='randomized', n_components=25, n_iter=3, random_state=42,
             tol=0.0)

In [352]:
for index,topic in enumerate(lsa_tfidf.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['city', 'percent', 'question', 'problem', 'love', 'build', 'system', 'school', 'kid', 'man', 'brain', 'country', 'story', 'child', 'woman']


THE TOP 15 WORDS FOR TOPIC #1
['boy', 'parent', 'love', 'community', 'education', 'kid', 'mother', 'country', 'story', 'family', 'school', 'child', 'girl', 'man', 'woman']


THE TOP 15 WORDS FOR TOPIC #2
['thank', 'piano', 'sing', 'video', 'instrument', 'hear', 'art', 'musician', 'piece', 'love', 'brain', 'song', 'sound', 'play', 'music']


THE TOP 15 WORDS FOR TOPIC #3
['memory', 'sleep', 'man', 'doctor', 'blood', 'gene', 'drug', 'disease', 'neuron', 'body', 'patient', 'cancer', 'woman', 'cell', 'brain']


THE TOP 15 WORDS FOR TOPIC #4
['girl', 'sun', 'animal', 'galaxy', 'space', 'story', 'ocean', 'star', 'light', 'man', 'water', 'planet', 'universe', 'earth', 'woman']


THE TOP 15 WORDS FOR TOPIC #5
['carbon', 'plant', 'city', 'percent', 'disease', 'patient', 'drug', 'climate', 'water', 'country', 'health', 'cance

In [342]:
import numpy as np
# Create Document - Topic Matrix
lsa_output = lsa_tfidf.transform(dtm_train_tfidf)

# column names
topicnames = ["Topic" + str(i) for i in range(lsa_tfidf.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(X_train))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lsa_output, 4), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Get next dominant topic for each document
sorted_topics = np.argsort(df_document_topic.values, axis = 1)
secondary_topic = [doc[-2] for doc in sorted_topics]
df_document_topic['seconary_topic'] = secondary_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [159]:
np.argmax(df_document_topic.values, axis=1)

array([ 0,  0,  0, ..., 25, 25,  0])

In [157]:
unique, counts = np.unique(dominant_topic, return_counts=True)

In [158]:
print(np.asarray((unique, counts)).T)

[[   0 2445]
 [   1   26]
 [   2  101]
 [   3   89]
 [   5    2]
 [   6   13]
 [   7    8]
 [   8   28]
 [  10   28]
 [  11   16]
 [  12   26]
 [  13    3]
 [  14    6]
 [  15    5]
 [  16    6]
 [  17    9]
 [  18    4]
 [  19   29]
 [  20    1]
 [  21   13]
 [  22    3]
 [  23    5]
 [  24   13]]


In [343]:
unique, counts = np.unique(secondary_topic, return_counts=True)

In [344]:
print(np.asarray((unique, counts)).T)

[[  1 324]
 [  2 215]
 [  3 276]
 [  4 222]
 [  5 128]
 [  6 159]
 [  7 153]
 [  8  72]
 [  9  54]
 [ 10 103]
 [ 11 114]
 [ 12 116]
 [ 13  78]
 [ 14  96]
 [ 15 114]
 [ 16 121]
 [ 17  87]
 [ 18  85]
 [ 19  52]
 [ 20  48]
 [ 21  65]
 [ 22  72]
 [ 23  72]
 [ 24  53]]


# Fit LDA

## CountVectorizer

## TfidfVectorizer

In [177]:
lda_tfidf = LatentDirichletAllocation(n_components=25,           # Number of topics
                                      max_iter=5,               # Max learning iterations
                                      learning_method='online',   
                                      random_state=42,          # Random state
                                      batch_size=100,            # n docs in each learning iter
                                      evaluate_every = -1,       # compute perplexity every n iters, default: Don't
                                      n_jobs = -1,               # Use all available CPUs
                                     )

In [178]:
lda_tfidf.fit_transform(dtm_train_tfidf)

array([[0.00297412, 0.00297412, 0.00297412, ..., 0.00297412, 0.92862114,
        0.00297412],
       [0.00470189, 0.00470189, 0.00470189, ..., 0.00470189, 0.88715454,
        0.00470189],
       [0.00448913, 0.00448913, 0.00448913, ..., 0.00448913, 0.89226077,
        0.00448913],
       ...,
       [0.00389272, 0.00389272, 0.00389272, ..., 0.00389272, 0.90657474,
        0.00389272],
       [0.00534758, 0.00534758, 0.00534758, ..., 0.00534758, 0.87165807,
        0.00534758],
       [0.00340726, 0.00340726, 0.00340726, ..., 0.00340726, 0.91822585,
        0.00340726]])

In [179]:
for index,topic in enumerate(lda_tfidf.components_):
    print(f'THE TOP 15 WORDS FOR TOPIC #{index}')
    print([tfidf.get_feature_names()[i] for i in topic.argsort()[-15:]])
    print('\n')

THE TOP 15 WORDS FOR TOPIC #0
['process', 'banjo', 'critical', 'light', 'play', 'chant', 'astrophysicist', 'clinical', 'extremist', 'limb', 'phantom', 'trial', 'cancer', 'music', 'map']


THE TOP 15 WORDS FOR TOPIC #1
['transcript', 'director', 'company', 'mobile', 'org', 'conversation', 'chart', 'phone', 'team', 'structure', 'gold', 'innovation', 'rhythm', 'technology', 'beat']


THE TOP 15 WORDS FOR TOPIC #2
['donor', 'arm', 'refine', 'flight', 'dilemma', 'augment', 'consensus', 'patient', 'scaffold', 'liver', 'group', 'transplant', 'stem', 'organ', 'cell']


THE TOP 15 WORDS FOR TOPIC #3
['oil', 'unit', 'planet', 'beetle', 'refrigeration', 'star', 'diplomat', 'antibiotic', 'posture', 'dung', 'fuel', 'cloud', 'joint', 'car', 'knee']


THE TOP 15 WORDS FOR TOPIC #4
['basic', 'labor', 'plank', 'beast', 'mountain', 'exile', 'person', 'door', 'mouth', 'chinese', 'fire', 'sun', 'tree', 'character', 'hercule']


THE TOP 15 WORDS FOR TOPIC #5
['young', 'prison', 'camp', 'bomb', 'suspicious'

In [180]:
# Create Document - Topic Matrix
lda_output = lda_tfidf.transform(dtm_train_tfidf)

# column names
topicnames = ["Topic" + str(i) for i in range(lda_tfidf.n_components)]

# index names
docnames = ["Doc" + str(i) for i in range(len(X_train))]

# Make the pandas dataframe
df_document_topic = pd.DataFrame(np.round(lda_output, 2), columns=topicnames, index=docnames)

# Get dominant topic for each document
dominant_topic = np.argmax(df_document_topic.values, axis=1)
df_document_topic['dominant_topic'] = dominant_topic

# Get next dominant topic for each document
sorted_topics = np.argsort(df_document_topic.values, axis = 1)
secondary_topic = [doc[-2] for doc in sorted_topics]
df_document_topic['seconary_topic'] = secondary_topic

# Styling
def color_green(val):
    color = 'green' if val > .1 else 'black'
    return 'color: {col}'.format(col=color)

def make_bold(val):
    weight = 700 if val > .1 else 400
    return 'font-weight: {weight}'.format(weight=weight)

# Apply Style
df_document_topics = df_document_topic.head(100).style.applymap(color_green).applymap(make_bold)
df_document_topics

In [144]:
unique, counts = np.unique(dominant_topic, return_counts=True)
print(np.asarray((unique, counts)).T)

[[   1 2871]
 [   8    6]
 [  11    2]]
